## Train ResNets

### created by Yuying Liu, 04/30/2020

This script is a template for training neural network time-steppers for different systems and different time scales. To reproduce the results in the paper, one needs to obtain all 11 neural network models for each nonlinear system under study. For setup details, please refer to Table 2 in the paper.

In [5]:
import os
import sys
import torch
import numpy as np

module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import ResNet as net

In [6]:
# adjustables

k = 10                        # model index: should be in {0, 2, ..., 10}
dt = 0.01                     # time unit: 0.0005 for Lorenz and 0.01 for others
system = 'Hyperbolic'         # system name: 'Hyperbolic', 'Cubic', 'VanDerPol', 'Hopf' or 'Lorenz'
noise = 0.0                   # noise percentage: 0.00, 0.01 or 0.02

lr = 1e-3                     # learning rate
max_epoch = 500000            # the maximum training epoch 
batch_size = 320              # training batch size
arch = [2, 128, 128, 128, 2]  # architecture of the neural network

In [12]:
# paths
data_dir = os.path.join('data/', system)
model_dir = os.path.join('models/', system)

# global const
n_forward = 5
ks = list(range(11))
step_size = 2**0#[2**k for k in ks]

In [13]:
# load data
train_data = np.load(os.path.join(data_dir, 'train_noise{}.npy'.format(noise)))
val_data = np.load(os.path.join(data_dir, 'val_noise{}.npy'.format(noise)))
test_data = np.load(os.path.join(data_dir, 'test_noise{}.npy'.format(noise)))
n_train = train_data.shape[0]
n_val = val_data.shape[0]
n_test = test_data.shape[0]

# create dataset object
dataset = net.DataSet(train_data, val_data, test_data, dt, step_size, n_forward)

In [14]:
model_name = 'model_D{}_noise{}.pt'.format(step_size, noise)

# create/load model object
try:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = torch.load(os.path.join(model_dir, model_name), map_location=device)
    model.device = device
except:
    print('create model {} ...'.format(model_name))
    model = net.ResNet(arch=arch, dt=dt, step_size=step_size)

# training
model.train_net(dataset, max_epoch=max_epoch, batch_size=batch_size, lr=lr,
                model_path=os.path.join(model_dir, model_name))

create model model_D1_noise0.0.pt ...
epoch 1000, training loss 4.034102971672837e-07, validation loss 5.656399935105583e-07
(--> new model saved @ epoch 1000)
epoch 2000, training loss 2.451866407682246e-07, validation loss 3.0321589861159737e-07
(--> new model saved @ epoch 2000)
epoch 3000, training loss 4.7545958636874275e-07, validation loss 5.787879899799009e-07
epoch 4000, training loss 4.7593735530426784e-07, validation loss 5.614136284748383e-07
epoch 5000, training loss 2.749507075350266e-07, validation loss 2.979831208449468e-07
(--> new model saved @ epoch 5000)
epoch 6000, training loss 8.04062452175458e-08, validation loss 1.0738859401726586e-07
(--> new model saved @ epoch 6000)
epoch 7000, training loss 1.4594854746974306e-06, validation loss 1.446823830519861e-06
epoch 8000, training loss 6.284055302785418e-07, validation loss 6.253319497773191e-07
epoch 9000, training loss 2.5177821498800768e-06, validation loss 2.5279121018684236e-06
epoch 10000, training loss 9.7166